In [5]:
%matplotlib notebook
import os
import h5py
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2

In [9]:
train = True

if train:
    path = test_path = os.path.abspath('../dataset/')    
else:
    path = train_path = os.path.abspath('../dataset/')

Read dataset from h5py file, there are three keys in the dataset, which we extract into a list of lists

In [12]:
# C:\Users\Andri\Documents\Luke\keras_Realtime_Multi-Person_Pose_Estimation\dataset\coco_train_dataset.h5
print(os.path.join(path, 'coco_train_dataset.h5'))
h5 = h5py.File(os.path.join(path, 'coco_train_dataset.h5')) # use coco_val_dataset.h5 for validation 

dataset, images, masks = [h5["dataset"], h5['images'], h5['masks']]

C:\Users\Andri\Documents\Luke\keras_Realtime_Multi-Person_Pose_Estimation\dataset\coco_train_dataset.h5


OSError: Unable to create file (unable to open file: name = 'C:\Users\Andri\Documents\Luke\keras_Realtime_Multi-Person_Pose_Estimation\dataset\coco_train_dataset.h5', errno = 17, error message = 'File exists', flags = 15, o_flags = 502)

In [ ]:
for key in dataset.keys():
    datadict = json.loads(dataset[key].value) # labeling for the image
    debug = json.loads(dataset[key].attrs['meta']) # attributes about the image
    print(datadict.keys())
    print(debug.keys())
    img = images[datadict['image']].value # get the image
    print(np.shape(img))
    
    mask_miss = None

    
    # conver the image using some magic... honestly what the fuck is going on here
    if len(img.shape)==2 and img.shape[1]==1:
        img = cv2.imdecode(img, flags=-1) 
        # somehow this converts a 1d array into the correct dimensions...

    if img.shape[2]>3: # idk if this ever gets hit
        mask_miss = img[:, :, 3]
        img = img[:, :, 0:3]
        
    if mask_miss is None:
        if masks is not None: # if we supplied masks
            mask_miss = masks[datadict['image']].value
            if len(mask_miss.shape) == 2 and mask_miss.shape[1]==1:
                mask_miss = cv2.imdecode(mask_miss, flags = -1)

    if mask_miss is None:
        mask_miss = np.full((img.shape[0], img.shape[1]), fill_value=255, dtype=np.uint8)
    break

In [ ]:
plt.imshow(img)
plt.imshow(mask_miss, alpha=.9)
plt.show()

In [ ]:
mask_miss

In [ ]:
import itertools
import matplotlib.cm as cm


joint_uvd = np.array(datadict['joints'])
jnt_uvd = np.squeeze(joint_uvd) # joint u, v, depth data
n = np.size(jnt_uvd,1) # number of joints
print(n)
print(np.shape(jnt_uvd))
jnt_scales = 20 *np.ones(n)
jnt_colors = itertools.cycle(cm.rainbow(np.linspace(0, 1, np.size(jnt_uvd,1)))) # random colors
plt.imshow(img) # display the image
# display the x, y, scale, and color of joints 
for person in jnt_uvd[:, :, 0:2]:
    for x, y in person:
        plt.scatter(x, y, color=next(jnt_colors))
plt.imshow(mask_miss, alpha=.7)
plt.show()

In [ ]:
np.size(jnt_uvd,1)

In [ ]:
for person in jnt_uvd[:, :, 0:2]:
    for x, y in person:
        print(x, y)
    break